# Problems:
* Decimal is unsupported -> conversion to float
* No constraints besides the unique constraints -> all checks should be performed on backend

In [11]:
from neo4j import GraphDatabase, DirectDriver

NEO_LOGIN = "neo4j"
NEO_PASSWORD = "bitnami"

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=(NEO_LOGIN, NEO_PASSWORD))




In [2]:
import psycopg2

PG_LOGIN = "DMD2user"
PG_PASS = "DMD2pgPass"
PG_DB = "dvdrental"

con = psycopg2.connect(database=PG_DB, user=PG_LOGIN,
                       password=PG_PASS, host="localhost", port="5432")

cursor = con.cursor()

In [2]:
cursor.execute("alter table store rename column manager_staff_id to staff_id")
con.commit()

In [13]:
from typing import List, Tuple, Optional
from decimal import Decimal

steps = 100
current_step = 1

def report(obj_name: str):
    global steps, current_step
    print(f"{obj_name} done: {current_step}/{steps}")
    current_step += 1

def make_label(s:str) -> str:
        return s.capitalize()
    
def make_var(s:str) -> str:
    return s.lower()

def fix_arg_type(arg):
    if type(arg) == Decimal:
        return float(arg)
    else:
        return arg

def transfer_table(driver: DirectDriver, cursor, tablename: str, *args, 
                   relationships: Optional[List[Tuple[str,str,str]]] = None,
                   indices:Optional[Tuple[str]] = None):
    label = make_label(tablename)
    var = make_var(tablename)
    insert_statement = f"CREATE ({var}: {label} {{ {', '.join([f'{prop}: ${prop}' for prop in args])} }})"
    sql_query = f"select {', '.join(args)} from {tablename}"
    unique = f"CREATE CONSTRAINT ON ({var}: {label}) ASSERT {var}.{tablename.lower()}_id IS UNIQUE"

    relation = "MATCH ({var_from}:{label_from}), ({var_to}:{label_to}) WHERE " \
               "{var_from}.{table_from}_id = ${table_from}_id  AND {var_to}.{table_to}_id = ${table_to}_id " \
               "CREATE ({var_from}) - [r:{relation_name}] -> ({var_to})" 
    
    remove_unused_id ="MATCH (n:{label_from})" \
                      "REMOVE n.{table_to}_id"
    
    cursor.execute(sql_query)
    with driver.session() as session:
        tx = session.begin_transaction()
        
        for row in cursor.fetchall():
            data = dict(zip(args, map(fix_arg_type, row)))
            tx.run(insert_statement, **data)
            
            if relationships is not None:
                for relationship in relationships:
                    tx.run(relation.format(
                        table_from=relationship[0], table_to=relationship[1],
                        var_from=make_var(relationship[0]), var_to=make_var(relationship[1]),
                        label_from=make_label(relationship[0]), label_to=make_label(relationship[1]),
                        relation_name=relationship[2]
                    ), **data)
                    
        if relationships is not None:
            for relationship in relationships:
                tx.run(remove_unused_id.format(label_from=make_label(relationship[0]),
                                               table_to=relationship[1]
                                               ))
        tx.sync()
        tx.commit()
        session.run(unique)
    
    report(tablename)
    if relationships is not None:
        for relationship in relationships:
            report(f"{relationship[0]} -> {relationship[1]}")
    
def transfer_many2many_rel(driver: DirectDriver, cursor, table_name: str, table_from:str, table_to: str, 
                           relation_name:str, *args):
    relation = "MATCH ({var_from}:{label_from}), ({var_to}:{label_to}) WHERE " \
               "{var_from}.{table_from}_id = ${table_from}_id  AND {var_to}.{table_to}_id = ${table_to}_id " \
               "CREATE ({var_from}) - [r:{relation_name}] -> ({var_to})".format(
                table_from=table_from, table_to=table_to,
                var_from=make_var(table_from), var_to=make_var(table_to),
                label_from=make_label(table_from), label_to=make_label(table_to),
                relation_name=relation_name
               )
    sql_query = f"select {', '.join(args)} from {table_name}"
    cursor.execute(sql_query)
    with driver.session() as session:
        tx = session.begin_transaction()
        for row in cursor.fetchall():
            data = dict(zip(args, map(fix_arg_type, row)))
            tx.run(relation, **data)
        tx.sync()
        tx.commit()
    report(f"{table_from} -> {table_to}")

def create_index(driver: DirectDriver, tablename:str, *fields:str):
    with driver.session() as session:
        session.run(f"CREATE INDEX ON :{make_label(tablename)} "
                    f"({', '.join(fields)})")

Connected to db
# Start transfer
## Category

In [6]:
transfer_table(driver, cursor, "category", "category_id", "name", "last_update")

category done: 1/100


## Language

In [7]:
transfer_table(driver, cursor, "language", "language_id", "name", "last_update")

language done: 2/100


## Film + Film-language connection

In [11]:
transfer_table(driver, cursor, "film", "film_id", "title", "description", "release_year", "rental_duration", "rental_rate", "length", 
"replacement_cost", "rating", "last_update", "special_features", "fulltext", "language_id",
               relationships=[("film", "language", "IN_LANGUAGE")])

film done: 1/100
film -> language done: 2/100


## Category-Film

In [14]:
transfer_many2many_rel(driver, cursor, "film_category", "film", "category", "IN_CATEGORY", 
                       "film_id", "category_id", "last_update")

film -> category done: 1/100


## Actor

In [15]:
transfer_table(driver, cursor, "actor", 
               "actor_id", "first_name", "last_name", "last_update")

actor done: 2/100


## Actor->Film connection

In [16]:
transfer_many2many_rel(driver, cursor, "film_actor", "actor", "film", "FILMED_IN", 
                       "actor_id", "film_id", "last_update")

actor -> film done: 3/100


## Inventory + film<-inventory

In [17]:
transfer_table(driver, cursor, "inventory", 
               "inventory_id", "film_id", "store_id", "last_update",
               relationships=[("inventory", "film", "RENTS_FILM")])

inventory done: 4/100
inventory -> film done: 5/100


## Country

In [18]:
transfer_table(driver, cursor, "country", "country_id", "country", "last_update")


country done: 6/100


## City

In [33]:
transfer_table(driver, cursor, "city",  
               "city_id", "city", "country_id", "last_update",
               relationships=[("city", "country", "SITUATED_IN")])

city done: 1/100
city -> country done: 2/100


## Address

In [34]:
transfer_table(driver, cursor, "address",
               "address_id", "address", "address2", "district", "city_id", "postal_code", "phone", "last_update",
               relationships=[("address", "city", "SITUATED_IN")])

address done: 3/100
address -> city done: 4/100


## Customer

In [35]:
transfer_table(driver, cursor, "customer",
               "customer_id", "store_id", "first_name", "last_name", "email", "address_id", "activebool",
               "create_date", "last_update", "active",
               relationships=[("customer", "address", "LIVES_AT")])

customer done: 5/100
customer -> address done: 6/100


## Staff

In [39]:
transfer_table(driver, cursor, "staff",
               "staff_id", "first_name", "last_name", "address_id", "email", "store_id", "active",
               "username", "password", "last_update",
               relationships=[("staff", "address", "LIVES_AT")])

staff done: 1/100
staff -> address done: 2/100


## Store

In [4]:
transfer_table(driver, cursor, "store", 
               "store_id", "staff_id", "address_id", "last_update",
               relationships=[("store", "staff", "MANAGED_BY"), ("store", "address", "LOCATED_AT")])

cursor.execute("alter table store rename column staff_id to manager_staff_id")
con.commit()

store done: 1/100
store -> staff done: 2/100
store -> address done: 3/100


## Rental
Very long step

In [6]:
transfer_table(driver, cursor, "rental",
               "rental_id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update",
               relationships=[("rental", "inventory", "RENTS"), 
                              ("rental", "customer", "RENTED_TO"),
                              ("rental", "staff", "RENTED_BY")])

rental done: 4/100
rental -> inventory done: 5/100
rental -> customer done: 6/100
rental -> staff done: 7/100


## Payment
Also long step

In [7]:
transfer_table(driver, cursor, "payment",
               "payment_id", "customer_id", "staff_id", "rental_id", "amount", "payment_date",
               relationships=[("payment", "customer", "PAID_BY"), 
                              ("payment", "staff", "ACCEPTED_BY"),
                              ("payment", "rental", "PAID_FOR")])

payment done: 8/100
payment -> customer done: 9/100
payment -> staff done: 10/100
payment -> rental done: 11/100
